![banner](https://github.com/hello-robot/stretch_mujoco/raw/main/docs/images/stretch_mujoco.png)

<h1><center>Getting Started Tutorial  <a href="https://colab.research.google.com/github/hello-robot/stretch_mujoco/blob/main/docs/getting_started.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" width="140" align="center"/></a></center></h1>

This notebook provides an introduction to Stretch's [**MuJoCo** simulation](https://github.com/hello-robot/stretch_mujoco/). You can run this notebook with either **CPU** or **GPU** instance.


## Install

In [1]:
# # Set up Stretch Mujoco repo
# # %rm -rf ./stretch_mujoco/
# # # !git clone https://github.com/hello-robot/stretch_mujoco --recurse-submodules
# # !git clone https://github.com/hello-robot/stretch_mujoco --depth 1
# # %cd ./stretch_mujoco/
# # %pip install -e ".[jupyter]"

# # Check if we can use GPU rendering
# import os
# import subprocess
# try:
#     subprocess.run('nvidia-smi')
#     USE_GPU=True
# except:
#     USE_GPU=False

# # Setup rendering
# if USE_GPU:
#     # Add an ICD config so that glvnd can pick up the Nvidia EGL driver.
#     # This is usually installed as part of an Nvidia driver package, but the Colab
#     # kernel doesn't install its driver via APT, and as a result the ICD is missing.
#     # (https://github.com/NVIDIA/libglvnd/blob/master/src/EGL/icd_enumeration.md)
#     NVIDIA_ICD_CONFIG_PATH = '/usr/share/glvnd/egl_vendor.d/10_nvidia.json'
#     if not os.path.exists(NVIDIA_ICD_CONFIG_PATH):
#         with open(NVIDIA_ICD_CONFIG_PATH, 'w') as f:
#             f.write("""{
#       "file_format_version" : "1.0.0",
#       "ICD" : {
#           "library_path" : "libEGL_nvidia.so.0"
#       }
#   }""")

#     # Configure MuJoCo to use the EGL rendering backend (requires GPU)
#     print('Setting environment variable to use GPU rendering:')
#     %env MUJOCO_GL=egl
# else:
#     # Required for OSMesa OpenGL driver
#     !apt-get update
#     !apt-get install -y libosmesa6-dev libgl1-mesa-glx libglfw3

#     print('Setting environment variable to use CPU rendering:')
#     %env MUJOCO_GL=osmesa

# # Other imports and helper functions
# import time
# import pprint
# import itertools
# import numpy as np

# # Graphics and plotting.
# !command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
# !pip install -q mediapy
# import mediapy as media
# import matplotlib.pyplot as plt

# # More legible printing from numpy.
# np.set_printoptions(precision=3, suppress=True, linewidth=100)

After installing, we need robocasa set up

In [2]:
# python -m pip install -e ".[robocasa]"
# python -m pip install -e third_party/robosuite
# python -m pip install -e third_party/robocasa

If you get an error like

(stretch_mujoco_v310) orrijoa@orrijoa-IdeaPad-Gaming-3-15ACH6:~/projects/stretch_mujoco_jupyter/stretch_mujoco$ python -m pip install -e third_party/robosuite
Obtaining file:///home/orrijoa/projects/stretch_mujoco_jupyter/stretch_mujoco/third_party/robosuite
ERROR: file:///home/orrijoa/projects/stretch_mujoco_jupyter/stretch_mujoco/third_party/robosuite does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.

From the root of your stretch_mujoco repo, run:
git submodule update --init


In [3]:
# python third_party/robosuite/robosuite/scripts/setup_macros.py
# python third_party/robocasa/robocasa/scripts/setup_macros.py
# python third_party/robocasa/robocasa/scripts/download_kitchen_assets.py

## Basics

The `StretchMujocoSimulator` class is used to:

 - Start/stop the simulation
 - Read camera imagery
 - Read lidar scans
 - Read joint states
 - Position control the robot's ranged joints
 - Velocity control the robot's mobile base

### Prerequisites to run to stream cameras in each windows

In [4]:
# Viewer on NVIDIA dGPU (windowed)
import os
os.environ.pop("MUJOCO_GL", None)              # ensure not headless
os.environ["__NV_PRIME_RENDER_OFFLOAD"] = "1"
os.environ["__GLX_VENDOR_LIBRARY_NAME"] = "nvidia"
os.environ["__VK_LAYER_NV_optimus"] = "NVIDIA_only"

# If you want headless instead, comment the above and use:
# import os, shutil
# os.environ["MUJOCO_GL"] = "egl" if shutil.which("nvidia-smi") else "osmesa"

In [5]:
import time
import threading
from pprint import pprint

import numpy as np
import matplotlib.pyplot as plt
import cv2

# More legible printing from numpy.
np.set_printoptions(precision=3, suppress=True, linewidth=100)

# Optional dependency
try:
    import mediapy as media
    HAS_MEDIAPY = True
except ImportError:
    HAS_MEDIAPY = False
    print("mediapy not installed. Install with: python -m pip install mediapy")
    
# Project imports last (after env is set!)
from stretch_mujoco import StretchMujocoSimulator
from stretch_mujoco.enums.stretch_sensors import StretchSensors
from stretch_mujoco.enums.stretch_cameras import StretchCameras
from stretch_mujoco.enums.actuators import Actuators

In [6]:
# # Quiet just the "Passive viewer... 30.0FPS" spam (bytes-safe)
# import sys, io, re

# _phrase = re.compile(r"Passive viewer and camera rendering is below the requested 30\.0FPS")

# class _StreamFilter(io.TextIOBase):
#     def __init__(self, wrapped):
#         self.wrapped = wrapped
#         self.encoding = getattr(wrapped, "encoding", "utf-8")

#     def write(self, s):
#         # accept both str and bytes
#         if isinstance(s, (bytes, bytearray)):
#             text = s.decode(self.encoding, errors="ignore")
#         else:
#             text = str(s)
#         # drop only the specific warning line
#         if _phrase.search(text):
#             return len(s) if isinstance(s, (bytes, bytearray)) else len(text)
#         return self.wrapped.write(text)

#     def flush(self):
#         return self.wrapped.flush()

# def mute_passive_fps_warning():
#     """Install the filter; returns restore() to undo it later."""
#     orig_out, orig_err = sys.stdout, sys.stderr
#     sys.stdout = _StreamFilter(sys.stdout)
#     sys.stderr = _StreamFilter(sys.stderr)
#     def restore():
#         nonlocal orig_out, orig_err
#         sys.stdout = orig_out
#         sys.stderr = orig_err
#     return restore


### Camera & Streaming Set Up

In [7]:
# keep track of which windows we've created
_created_windows = set()

def show_camera_feeds_sync(sim, print_fps=False, init_size=(640, 480)):
    """
    Pull camera data from the simulator and display it using OpenCV.
    Windows are resizable (drag corners).
    """
    camera_data = sim.pull_camera_data()

    if print_fps:
        s = sim.pull_status()
        print(f"Physics fps: {s.fps}. Camera FPS: {camera_data.fps}. {s.sim_to_real_time_ratio_msg}")

    for cam_enum, pixels in camera_data.get_all(use_depth_color_map=True).items():
        if pixels is None:
            continue

        name = cam_enum.name  # window title

        # create a resizable window once per camera
        if name not in _created_windows:
            cv2.namedWindow(name, cv2.WINDOW_NORMAL)       # <-- resizable
            cv2.resizeWindow(name, *init_size)             # initial size
            # optional: keep aspect ratio when resizing
            try:
                cv2.setWindowProperty(name, cv2.WND_PROP_ASPECT_RATIO, cv2.WINDOW_KEEPRATIO)
            except Exception:
                pass
            _created_windows.add(name)

        cv2.imshow(name, pixels)

    # process window events (needed for resizing to take effect)
    cv2.waitKey(1)
    
_stream_evt = None
_stream_thread = None

def start_stream_thread(sim, print_fps=False, target_hz=20):
    """Run camera streaming in the background using your show_camera_feeds_sync()."""
    global _stream_evt, _stream_thread
    if _stream_thread and _stream_thread.is_alive():
        print("Stream already running.")
        return

    _stream_evt = threading.Event()

    def _worker():
        try:
            dt = 1.0 / max(1, target_hz)
            while not _stream_evt.is_set() and sim.is_running():
                show_camera_feeds_sync(sim, print_fps)
                # sim.step()                       # advance physics
                time.sleep(dt)                   # throttle display FPS a bit
        except Exception as e:
            print("stream thread ended:", type(e).__name__, e)
        finally:
            # Close any OpenCV windows cleanly
            try:
                cv2.destroyAllWindows()
            except: 
                pass

    _stream_thread = threading.Thread(target=_worker, daemon=True)
    _stream_thread.start()
    print("Stream thread started.")

def stop_stream_thread():
    """Stop the background stream cleanly (call before sim.stop())."""
    global _stream_evt, _stream_thread
    if _stream_evt:
        _stream_evt.set()
    if _stream_thread:
        _stream_thread.join(timeout=2.0)
    try:
        cv2.waitKey(1)
        cv2.destroyAllWindows()
    except:
        pass
    _stream_evt = None
    _stream_thread = None
    print("Stream thread stopped.")


### Testing for Robocasa Set up

In [8]:
# Core libs
import mujoco
import numpy as np

# RoboCasa generator
try:
    from stretch_mujoco.robocasa_gen import model_generation_wizard
    print("Found model_generation_wizard()")
except Exception as e:
    print("Could not import model_generation_wizard:", e)

# robosuite / robocasa sanity
import robosuite
print("robosuite version:", getattr(robosuite, "__version__", "unknown"))

import robocasa
print("robocasa version:", getattr(robocasa, "__version__", "unknown"))

# Show robosuite macro backend if present
try:
    from robosuite import macros as RS_MACROS
    print("robosuite MUJOCO_GL:", getattr(RS_MACROS, "MUJOCO_GL", "not set"))
except Exception as e:
    print("robosuite macros import issue:", e)


[robosuite WARNING] Could not import robosuite_models. Some robots may not be available. If you want to use these robots, please install robosuite_models from source (https://github.com/ARISE-Initiative/robosuite_models) or through pip install. (__init__.py:30)
[robosuite WARNING] Could not load the mink-based whole-body IK. Make sure you install related import properly, otherwise you will not be able to use the default IK controller setting for GR1 robot. (__init__.py:40)


Found model_generation_wizard()
robosuite version: 1.5.1
robocasa version: 0.2.0
robosuite MUJOCO_GL: not set


### Set Up Mujoco Env

In [9]:
model = xml = objects_info = env = None

try:
    # Non interactive example. Adjust task/layout/style later if you want.
    # These names are common defaults; if they ever change, the except block will let you pick via wizard.
    model, xml, objects_info, env = model_generation_wizard(
        task="PnPCounterToCab",
        layout=0,
        style=0
    )
    print("Generated RoboCasa model non-interactively.")
except TypeError:
    # Some versions use only the interactive wizard
    print("Non-interactive args not supported. Opening interactive wizard...")
    model, xml, objects_info, env = model_generation_wizard()
except FileNotFoundError as e:
    print("Asset missing:", e)
    print("Re-run the RoboCasa asset downloader script and try again.")
    raise
except Exception as e:
    print("RoboCasa scene generation failed:", e)
    raise

print("Model OK:", isinstance(model, mujoco.MjModel))
if xml:
    print("XML length:", len(xml))
if objects_info is not None:
    # objects_info is usually a dict from the generator
    print("Objects info keys:", list(objects_info)[:5])

env.close()


[robosuite INFO] Loading controller configuration from: /home/orrijoa/projects/stretch_mujoco_jupyter/stretch_mujoco/third_party/robosuite/robosuite/controllers/config/robots/default_pandaomron.json (composite_controller_factory.py:121)


Initializing environment...
Initial observation keys: odict_keys(['robot0_joint_pos_cos', 'robot0_joint_pos_sin', 'robot0_joint_vel', 'robot0_eef_pos', 'robot0_eef_quat', 'robot0_eef_quat_site', 'robot0_gripper_qpos', 'robot0_gripper_qvel', 'robot0_base_pos', 'robot0_base_quat', 'robot0_base_to_eef_pos', 'robot0_base_to_eef_quat', 'robot0_base_to_eef_quat_site', 'apple0_pos', 'apple0_quat', 'apple0_to_robot0_eef_pos', 'apple0_to_robot0_eef_quat', 'avocado0_pos', 'avocado0_quat', 'avocado0_to_robot0_eef_pos', 'avocado0_to_robot0_eef_quat', 'banana0_pos', 'banana0_quat', 'banana0_to_robot0_eef_pos', 'banana0_to_robot0_eef_quat', 'robot0_proprio-state', 'object-state'])
env.object_cfgs after override:
0: name=apple0      cat=apple       model=/home/orrijoa/projects/stretch_mujoco_jupyter/stretch_mujoco/third_party/robocasa/robocasa/models/assets/objects/objaverse/apple/apple_0/model.xml
1: name=avocado0    cat=avocado     model=/home/orrijoa/projects/stretch_mujoco_jupyter/stretch_mujoco/

In [10]:
# names = [n for n in dir(env) if not n.startswith("_")]
# methods = sorted(n for n in names if callable(getattr(env, n, None)))
# attrs = sorted(set(names) - set(methods))

# env.

In [11]:
# action = np.random.randn(*env.action_spec[0].shape) * 0.1
# obs, reward, done, info = env.step(action)
# obs

In [12]:
# Pretty-print objects and their initial placements
for body_name, info in objects_info.items():
    print(f"{body_name:30s}  cat={info['cat']:12s}  pos={info['pos']}  quat={info['quat']}")

apple0_main                     cat=apple         pos=(2.365647948610641, -0.4264353781428943, 0.96314560905)  quat=[0.948 0.    0.    0.318]
avocado0_main                   cat=avocado       pos=(2.0538701301593534, -0.20255546045583106, 0.9697274932800001)  quat=[ 0.971  0.     0.    -0.24 ]
banana0_main                    cat=banana        pos=(2.123160947646826, -0.14310175980215695, 1.4454152821139667)  quat=[0.985 0.    0.    0.175]


### Actual Testing with Tele Ops

In [ ]:
# sim = StretchMujocoSimulator(cameras_to_use=StretchCameras.all())

cameras_to_use = []
# cameras_to_use = StretchCameras.all()


sim = StretchMujocoSimulator(model=model, cameras_to_use=cameras_to_use)
sim.start(headless=False)

Starting Stretch Mujoco Simulator...
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
ERROR: could not create window

Press Enter to exit ...Still waiting to connect to the Mujoco Simulatior.


/home/orrijoa/miniconda3/envs/stretch_mujoco_v310/lib/python3.10/site-packages/glfw/__init__.py:917: GLFWError: (65543) b'GLX: Failed to create context: BadValue (integer parameter out of range for operation)'
  warnings.warn(message, GLFWError)


Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the Mujoco Simulatior.
Still waiting to connect to the

In [ ]:
# import time

# # 1) Register all objects at once using the exact names from objects_info
# names = list(objects_info.keys())  # e.g., ["apple0_main", "avocado0_main", "banana0_main", ...]
# sim.data_proxies.set_tracked_objects(names)

# # 2) Give the server a few ticks to publish poses, then fetch
# t, mapping = 0.0, {}
# for _ in range(10):
#     t, mapping = sim.data_proxies.get_objects_state()
#     if mapping:
#         break
#     time.sleep(0.02)

# print(f"objects={len(mapping)} at t={t:.3f}")

# import types, numpy as np, time

# def pull_all_objects_state(self):
#     # First try current snapshot
#     t, mapping = self.data_proxies.get_objects_state()
#     if not mapping:
#         # Lazy register all known object bodies from objects_info
#         try:
#             self.data_proxies.set_tracked_objects(list(objects_info.keys()))
#         except Exception:
#             pass
#         # Allow a few server ticks to publish
#         for _ in range(10):
#             t, mapping = self.data_proxies.get_objects_state()
#             if mapping:
#                 break
#             time.sleep(0.02)
#     # Return numpy arrays (pos: (3,), quat: (4,))
#     return {k: {"pos": np.array(v[0], dtype=float), "quat": np.array(v[1], dtype=float)} for k, v in mapping.items()}

# sim.pull_all_objects_state = types.MethodType(pull_all_objects_state, sim)

# # Use it
# state = sim.pull_all_objects_state()
# print(f"Got {len(state)} objects.")

# def print_objects_state(state):
#     for name in sorted(state):
#         pos = state[name]["pos"]; quat = state[name]["quat"]
#         cat = objects_info.get(name, {}).get("cat", "?")
#         print(f"{name:30s}  cat={cat:12s}  pos=({pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f})  quat=[ {quat[0]:.3f} {quat[1]:.3f} {quat[2]:.3f} {quat[3]:.3f} ]")

# print_objects_state(state)

In [ ]:
sim.register_tracked_objects(list(objects_info.keys()))
state = sim.pull_objects_state()

print(f"Got {len(state)} objects.")

def print_objects_state(state):
    for name in sorted(state):
        pos = state[name]["pos"]; quat = state[name]["quat"]
        cat = objects_info.get(name, {}).get("cat", "?")
        print(f"{name:30s}  cat={cat:12s}  pos=({pos[0]:.3f}, {pos[1]:.3f}, {pos[2]:.3f})  quat=[ {quat[0]:.3f} {quat[1]:.3f} {quat[2]:.3f} {quat[3]:.3f} ]")

print_objects_state(state)

In [ ]:
# run_stream_on_main(sim, print_fps=False, target_hz=20, enable_wasd=True)

In [ ]:
# restore_out = mute_passive_fps_warning()

In [ ]:
start_stream_thread(sim, print_fps=False, target_hz=20)

### ROBOT MANIPULATION

In [ ]:
sim.move_by(Actuators.head_tilt, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.head_tilt, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.head_pan, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.head_pan, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.base_translate, 0.1)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.base_translate, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.base_rotate, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.base_rotate, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.lift, 1.1)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.lift, -1.0)
time.sleep(0.5)

In [ ]:
sim.move_to(Actuators.arm, 0.0)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.arm, 0.05)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_yaw, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_yaw, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_pitch, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_pitch, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_roll, 0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.wrist_roll, -0.2)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.gripper, 0.07)
time.sleep(0.5)

In [ ]:
sim.move_by(Actuators.gripper, -0.07)
time.sleep(0.5)

### Report current status and limits

In [ ]:
pprint(sim.pull_status())

In [ ]:
pprint(sim.pull_camera_data())



In [ ]:
# radar related
pprint(sim.pull_sensor_data())

In [ ]:
print(sim.pull_joint_limits())

### End the simulation

In [ ]:
# # Try to undo any previous manual wrapping if you still have the originals
# try:
#     sys.stdout = _sys_stdout_orig
#     sys.stderr = _sys_stderr_orig
# except NameError:
#     pass

# 1) stop background streaming first
stop_stream_thread()

# 2) give the sim process a short moment to finish its own threads
time.sleep(0.1)

# 3) stop the simulator
if sim.is_running():
    sim.stop()

# 4) as a final sweep, make sure no OpenCV windows remain
try:
    cv2.waitKey(1)
    cv2.destroyAllWindows()
except:
    pass
